In [1]:
import torch.nn.functional as F
import torch.nn as nn
class UpSample1d(nn.Module):
    def __init__(self, layer_type):
        super().__init__()
        self.layer_type = layer_type

    def forward(self, x):
        if self.layer_type == 'none':
            return x
        else:
            return F.interpolate(x, scale_factor=2, mode='nearest')

In [2]:
import torch
test_upsample = UpSample1d(True)
data = torch.ones([1,1,10])
res = test_upsample(data)
print(res.shape)
print(res)
print(data)

In [1]:
from torch.utils.tensorboard import SummaryWriter

class AdaIN1d(nn.Module):
    def __init__(self, style_dim, num_features):
        super().__init__()
        self.norm = nn.InstanceNorm1d(num_features, affine=False)
        self.fc = nn.Linear(style_dim, num_features*2)

    def forward(self, x, s):
        h = self.fc(s)
        h = h.view(h.size(0), h.size(1), 1)
        gamma, beta = torch.chunk(h, chunks=2, dim=1)
        return (1 + gamma) * self.norm(x) + beta

model = AdaIN1d(style_dim=64, num_features=16)

# Tạo đầu vào giả: x là tensor (batch_size=1, num_features=16, sequence_length=64)
# s là tensor (batch_size=1, style_dim=64)
x = torch.randn(1, 16, 64)  # Đầu vào dữ liệu x
s = torch.randn(1, 64)      # Đầu vào style vector s
writer = SummaryWriter('runs/simple_model_experiment')

# Ghi lại kiến trúc của mô hình vào TensorBoard
writer.add_graph(model, (x,s))
writer.close()

print("Mô hình đã được ghi vào TensorBoard.")